In [1]:
import os
%pwd

'/media/sasan/Common/ML/EtoE/research'

In [2]:
os.chdir('../')
%pwd

'/media/sasan/Common/ML/EtoE'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from RedWineQualityPrediction.constanst import *
from RedWineQualityPrediction.utils.common import read_yaml, create_derectories

In [54]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_derectories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_derectories([config.root_dir])
        
        return ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.keys()
        )

In [30]:
import pandas as pd
import os
from RedWineQualityPrediction import logger
from sklearn.linear_model import ElasticNet
import joblib

In [61]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def training(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        target_column = list(self.config.target_column)[0]
        
        train_x = train_data.drop(columns=[target_column], axis=1)
        test_x = test_data.drop(columns=[target_column], axis=1)
        train_y = train_data[target_column]
        test_y = test_data[target_column]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        


In [63]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.training()
except Exception as error:
    raise error

[2025-02-10 23:52:25,340: INFO: common: Reading the yaml file from ::     config/config.yml]
[2025-02-10 23:52:25,344: INFO: common: Reading the yaml file from ::     params.yml]
[2025-02-10 23:52:25,351: INFO: common: Reading the yaml file from ::     schema.yml]
[2025-02-10 23:52:25,353: INFO: common: Directory created at :: artifacts]
[2025-02-10 23:52:25,355: INFO: common: Directory created at :: artifacts/model_trainer]
